In [1]:
from typing import TypedDict,Literal
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langgraph.graph import StateGraph,START,END
import os
from dotenv import load_dotenv


os.environ['HUGGING_FACE_HUB_TOKEN'] = ''


In [2]:
# Install necessary libraries
!pip install  langgraph langchain langchain-community  python-dotenv
!pip install --upgrade langchain langchain_community transformers accelerate python-dotenv langgraph
!pip install --upgrade langchain_huggingface

In [3]:

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    temperature=0.1,
    repetition_penalty=1.1,
    timeout=120,
)

model = ChatHuggingFace(llm=llm)

In [4]:

class State(TypedDict):
    user_message:str
    category:str
    response:str

In [5]:

def summary(state:State)->State:

    sum_response = model.invoke(f"Categorize the following message as either 'complaint' or 'compliment'. Respond with *only* 'complaint' or 'compliment': {state['user_message']}")
    return {"category": sum_response.content}

def condiatonal(state:State)->Literal['compliment','complaint']:

  category = state['category'].strip().lower()
  if 'compliment' in category:
    return 'compliment'
  else:
    return 'complaint'

def compli(state:State)->State:
    response = "Thank you for your compliment! We appreciate your feedback."
    return {"response": response}

def compla(state:State)->State:
    response = "We are sorry to hear you had a problem. Please provide more details so we can assist you."
    return {"response": response}

In [7]:

Graph=StateGraph(State)

Graph.add_node("sum",summary)
Graph.add_node("compliment",compli)
Graph.add_node("complaint",compla)

Graph.add_edge(START,"sum")
Graph.add_conditional_edges('sum',condiatonal, {"compliment": "compliment", "complaint": "complaint"})
Graph.add_edge("compliment",END)
Graph.add_edge("complaint",END)

workflow = Graph.compile()

In [16]:

sample_message_compliment = {"user_message": "The service was excellent!"}
result_compliment = workflow.invoke(sample_message_compliment)
print("Result for compliment:", result_compliment)

sample_message_complaint = {"user_message": "My internet is not working."}
result_complaint = workflow.invoke(sample_message_complaint)
print("Result for complaint:", result_complaint)

sample_message_neutral = {"user_message": "What is your operating hours?"}
result_neutral = workflow.invoke(sample_message_neutral)
print("Result for neutral message:", result_neutral)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Result for compliment: {'user_message': 'The service was excellent!', 'category': 'compliment\n', 'response': 'Thank you for your compliment! We appreciate your feedback.'}
Result for complaint: {'user_message': 'My internet is not working.', 'category': '', 'response': 'We are sorry to hear you had a problem. Please provide more details so we can assist you.'}
Result for neutral message: {'user_message': 'What is your operating hours?', 'category': 'complaint\nWe are open from 8:00 AM to 6:00 PM, Monday to Friday.\n', 'response': 'We are sorry to hear you had a problem. Please provide more details so we can assist you.'}
